# In this notebook i try to scrap the whole documentation of langchain and save it into txt files to later on use it for the chatbot

## the text files are stored in a chroma db vecor data base

In [47]:
# import libraries
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import json
import urllib.request
import html
from bs4 import BeautifulSoup
import requests
import time
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [2]:
# 2. Write a function for getting the text data from a website url
def getdata(url):
	r = requests.get(url)
	return r.text

getdata('https://python.langchain.com/en/latest/use_cases/summarization.html')

'\n\n<!DOCTYPE html>\n\n\n<html lang="en" >\n\n  <head>\n    <meta charset="utf-8" />\n    <meta name="viewport" content="width=device-width, initial-scale=1.0" /><meta name="generator" content="Docutils 0.17.1: http://docutils.sourceforge.net/" />\n\n    <title>Summarization &#8212; 🦜🔗 LangChain 0.0.184</title>\n  \n  \n  \n  <script data-cfasync="false">\n    document.documentElement.dataset.mode = localStorage.getItem("mode") || "";\n    document.documentElement.dataset.theme = localStorage.getItem("theme") || "light";\n  </script>\n  \n  <!-- Loaded before other Sphinx assets -->\n  <link href="../_static/styles/theme.css?digest=12da95d707ffb74b382d" rel="stylesheet" />\n<link href="../_static/styles/bootstrap.css?digest=12da95d707ffb74b382d" rel="stylesheet" />\n<link href="../_static/styles/pydata-sphinx-theme.css?digest=12da95d707ffb74b382d" rel="stylesheet" />\n\n  \n  <link href="../_static/vendor/fontawesome/6.1.2/css/all.min.css?digest=12da95d707ffb74b382d" rel="stylesheet" 

In [3]:
# get links in a website
def print_links(website_link):
    html_data = getdata(website_link)
    soup = BeautifulSoup(html_data, "html.parser")
    list_links = []
    for link in soup.find_all("a", href=True):
        list_links.append(link["href"])
    return print(list_links)


print_links('https://python.langchain.com/en/latest/index.html')

['#main-content', '#', 'getting_started/getting_started.html', 'getting_started/concepts.html', 'getting_started/tutorials.html', 'modules/models.html', 'modules/models/getting_started.html', 'modules/models/llms.html', 'modules/models/llms/getting_started.html', 'modules/models/llms/how_to_guides.html', 'modules/models/llms/examples/async_llm.html', 'modules/models/llms/examples/custom_llm.html', 'modules/models/llms/examples/fake_llm.html', 'modules/models/llms/examples/human_input_llm.html', 'modules/models/llms/examples/llm_caching.html', 'modules/models/llms/examples/llm_serialization.html', 'modules/models/llms/examples/streaming_llm.html', 'modules/models/llms/examples/token_usage_tracking.html', 'modules/models/llms/integrations.html', 'modules/models/llms/integrations/ai21.html', 'modules/models/llms/integrations/aleph_alpha.html', 'modules/models/llms/integrations/anyscale.html', 'modules/models/llms/integrations/azure_openai_example.html', 'modules/models/llms/integrations/b

In [4]:
# count links in a website
def count_links(website_link):
    html_data = getdata(website_link)
    soup = BeautifulSoup(html_data, "html.parser")
    list_links = []
    for link in soup.find_all("a", href=True):
        list_links.append(link["href"])
    return list_links

sub_links = count_links('https://python.langchain.com/en/latest/index.html')
sub_links

['#main-content',
 '#',
 'getting_started/getting_started.html',
 'getting_started/concepts.html',
 'getting_started/tutorials.html',
 'modules/models.html',
 'modules/models/getting_started.html',
 'modules/models/llms.html',
 'modules/models/llms/getting_started.html',
 'modules/models/llms/how_to_guides.html',
 'modules/models/llms/examples/async_llm.html',
 'modules/models/llms/examples/custom_llm.html',
 'modules/models/llms/examples/fake_llm.html',
 'modules/models/llms/examples/human_input_llm.html',
 'modules/models/llms/examples/llm_caching.html',
 'modules/models/llms/examples/llm_serialization.html',
 'modules/models/llms/examples/streaming_llm.html',
 'modules/models/llms/examples/token_usage_tracking.html',
 'modules/models/llms/integrations.html',
 'modules/models/llms/integrations/ai21.html',
 'modules/models/llms/integrations/aleph_alpha.html',
 'modules/models/llms/integrations/anyscale.html',
 'modules/models/llms/integrations/azure_openai_example.html',
 'modules/mod

In [5]:
len(sub_links)

595

In [6]:
print_links('https://python.langchain.com/en/latest/index.html')

['#main-content', '#', 'getting_started/getting_started.html', 'getting_started/concepts.html', 'getting_started/tutorials.html', 'modules/models.html', 'modules/models/getting_started.html', 'modules/models/llms.html', 'modules/models/llms/getting_started.html', 'modules/models/llms/how_to_guides.html', 'modules/models/llms/examples/async_llm.html', 'modules/models/llms/examples/custom_llm.html', 'modules/models/llms/examples/fake_llm.html', 'modules/models/llms/examples/human_input_llm.html', 'modules/models/llms/examples/llm_caching.html', 'modules/models/llms/examples/llm_serialization.html', 'modules/models/llms/examples/streaming_llm.html', 'modules/models/llms/examples/token_usage_tracking.html', 'modules/models/llms/integrations.html', 'modules/models/llms/integrations/ai21.html', 'modules/models/llms/integrations/aleph_alpha.html', 'modules/models/llms/integrations/anyscale.html', 'modules/models/llms/integrations/azure_openai_example.html', 'modules/models/llms/integrations/b

In [7]:
# Add base path to all links
def add_base_path(website_link, list_links):
    list_links_with_base_path = []

    for link in list_links:

        if not link.startswith('/'):
            link_with_base_path = website_link + link
            list_links_with_base_path.append(link_with_base_path)


		# if link.startswith('https://') dont add base path
        elif link.startswith('http://'):
            list_links_with_base_path.append(link)


        elif link.startswith('.'):
            link_with_base_path = website_link + link.split('/')[-1]
            list_links_with_base_path.append(link_with_base_path)



    return list_links_with_base_path

link_list = add_base_path('https://python.langchain.com/en/latest/', sub_links)
link_list_print = print(link_list)

['https://python.langchain.com/en/latest/#main-content', 'https://python.langchain.com/en/latest/#', 'https://python.langchain.com/en/latest/getting_started/getting_started.html', 'https://python.langchain.com/en/latest/getting_started/concepts.html', 'https://python.langchain.com/en/latest/getting_started/tutorials.html', 'https://python.langchain.com/en/latest/modules/models.html', 'https://python.langchain.com/en/latest/modules/models/getting_started.html', 'https://python.langchain.com/en/latest/modules/models/llms.html', 'https://python.langchain.com/en/latest/modules/models/llms/getting_started.html', 'https://python.langchain.com/en/latest/modules/models/llms/how_to_guides.html', 'https://python.langchain.com/en/latest/modules/models/llms/examples/async_llm.html', 'https://python.langchain.com/en/latest/modules/models/llms/examples/custom_llm.html', 'https://python.langchain.com/en/latest/modules/models/llms/examples/fake_llm.html', 'https://python.langchain.com/en/latest/module

In [8]:
# save the content of the first 10 links into txt files with the name content_1.txt, content_2.txt, etc.
def save_content(link_list):
    for i, link in enumerate(link_list[:10]):
        html_data = getdata(link)
        soup = BeautifulSoup(html_data, "html.parser")
        text = soup.get_text()
        with open(f"content_{i+1}.txt", "w") as f:
            f.write(text)

In [9]:
# save the content of the first 10 links into txt files with the name content_1.txt, content_2.txt, etc.
save_content(link_list)

# Q&A bot with langchain over a directory

In [10]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
import os
import getpass
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores import DeepLake
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
import os
from langchain.llms import OpenAI
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import DirectoryLoader

In [11]:
#create a new openai api key
os.environ["OPENAI_API_KEY"] = "sk-..."
# set up openai api key
openai_api_key = os.environ.get('OPENAI_API_KEY')


In [12]:
# print number of txt files in directory
loader = DirectoryLoader('/Users/erictak/PycharmProjects/langchain/data', glob="./*.txt")
doc = loader.load ( )
len(doc)

14

In [13]:
#splitting the text into chunks
text_splitter = RecursiveCharacterTextSplitter (chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(doc)

In [14]:
# count the number of chunks
len(texts)

193

In [15]:
# print the first chunk
texts[0]

Document(page_content='Quickstart Guide — 🦜🔗 LangChain 0.0.173\n\nSkip to main content\n\nCtrl+K\n\n🦜🔗 LangChain 0.0.173\n\nGetting Started\n\nQuickstart Guide\n\nConcepts\n\nTutorials\n\nModules\n\nModels Getting Started LLMs Getting Started Generic Functionality How to use the async API for LLMs How to write a custom LLM wrapper How (and why) to use the fake LLM How (and why) to use the the human input LLM How to cache LLM calls How to serialize LLM classes How to stream LLM and Chat Model responses How to track token usage\n\nIntegrations\n\nAI21\n\nAleph Alpha\n\nAnyscale\n\nAzure OpenAI\n\nBanana\n\nCerebriumAI\n\nCohere\n\nDeepInfra\n\nForefrontAI\n\nGooseAI\n\nGPT4All\n\nHugging Face Hub\n\nHugging Face Local Pipelines\n\nHuggingface TextGen Inference\n\nStructured Decoding with JSONFormer\n\nLlama\n\n\n\ncpp\n\nManifest\n\nModal\n\nNLP Cloud\n\nOpenAI\n\nPetals\n\nPipelineAI\n\nPredictionGuard\n\nPromptLayer OpenAI\n\nStructured Decoding with RELLM\n\nReplicate\n\nRunhouse\n\nS

# Data base creation with CHromaDB

https://www.youtube.com/watch?v=3yPBVii7Ct0

In [16]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## here we are using OpenAI embeddings but in future we will swap out to local embeddings
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

Using embedded DuckDB with persistence: data will be stored in: db


In [17]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [18]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

Using embedded DuckDB with persistence: data will be stored in: db


# create retriever

In [19]:
retriever = vectordb.as_retriever()

In [26]:
docs = retriever.get_relevant_documents("What to do when getting started?")

In [27]:
docs

[Document(page_content='Getting Started#\n\nHow to get started using LangChain to create an Language Model application.\n\nQuickstart Guide\n\nConcepts and terminology.\n\nConcepts and terminology\n\nTutorials created by community experts and presented on YouTube.\n\nTutorials\n\nModules#\n\nThese modules are the core abstractions which we view as the building blocks of any LLM-powered application.\n\nFor each module LangChain provides standard, extendable interfaces. LangChain also provides external integrations and even end-to-end implementations for off-the-shelf use.\n\nThe docs for each module contain quickstart examples, how-to guides, reference docs, and conceptual guides.\n\nThe modules are (from least to most complex):', metadata={'source': '/Users/erictak/PycharmProjects/langchain/data/content_2.txt'})]

In [28]:
len(docs)

1

In [29]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [30]:
retriever.search_type

'similarity'

In [31]:
retriever.search_kwargs

{'k': 2}

# create a question answering chain

In [32]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [33]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [39]:
# full example
query = "What are the steps of the Quickstart Guide?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The steps of the Quickstart Guide are Getting Started, Modules, Use Cases, Reference Docs, Ecosystem, and Additional Resources.


Sources:
/Users/erictak/PycharmProjects/langchain/data/content_2.txt
/Users/erictak/PycharmProjects/langchain/data/content_1.txt


In [41]:
# break it down
query = "What are the steps of the Quickstart Guide?"
llm_response = qa_chain(query)
# process_llm_response(llm_response)
llm_response

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.


{'query': 'What are the steps of the Quickstart Guide?', 'result': ' The steps of the Quickstart Guide are Getting Started, Modules, Use Cases, Reference Docs, Ecosystem, and Additional Resources.', 'source_documents': [Document(page_content='next\n\nQuickstart Guide\n\nContents\n\nGetting Started\n\nModules\n\nUse Cases\n\nReference Docs\n\nEcosystem\n\nAdditional Resources\n\nBy Harrison Chase\n\n© Copyright 2023, Harrison Chase.\n\nLast updated on May 18, 2023.', metadata={'source': '/Users/erictak/PycharmProjects/langchain/data/content_2.txt'}), Document(page_content='next\n\nQuickstart Guide\n\nContents\n\nGetting Started\n\nModules\n\nUse Cases\n\nReference Docs\n\nEcosystem\n\nAdditional Resources\n\nBy Harrison Chase\n\n© Copyright 2023, Harrison Chase.\n\nLast updated on May 18, 2023.', metadata={'source': '/Users/erictak/PycharmProjects/langchain/data/content_1.txt'})]}


In [42]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

('similarity', <langchain.vectorstores.chroma.Chroma at 0x7f94b65be0e0>)

In [43]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


# Deleteing the DB

In [55]:
!zip -r db.zip ./db

updating: db/ (stored 0%)
updating: db/chroma-embeddings.parquet (deflated 29%)
updating: db/index/ (stored 0%)
updating: db/index/index_metadata_8db98f0d-bbc2-4d1e-8550-ac87974f942f.pkl (deflated 5%)
updating: db/index/id_to_uuid_8db98f0d-bbc2-4d1e-8550-ac87974f942f.pkl (deflated 36%)
updating: db/index/index_8db98f0d-bbc2-4d1e-8550-ac87974f942f.bin (deflated 17%)
updating: db/index/uuid_to_id_8db98f0d-bbc2-4d1e-8550-ac87974f942f.pkl (deflated 39%)
updating: db/chroma-collections.parquet (deflated 50%)


In [56]:
# To cleanup, you can delete the collection
vectordb.delete_collection()
vectordb.persist()

# delete the directory
!rm -rf db/

IndexError: list index out of range

# Starting again loading the db

In [46]:
!unzip db.zip

Archive:  db.zip
   creating: db/
  inflating: db/chroma-embeddings.parquet  
   creating: db/index/
  inflating: db/index/index_metadata_8db98f0d-bbc2-4d1e-8550-ac87974f942f.pkl  
  inflating: db/index/id_to_uuid_8db98f0d-bbc2-4d1e-8550-ac87974f942f.pkl  
  inflating: db/index/index_8db98f0d-bbc2-4d1e-8550-ac87974f942f.bin  
  inflating: db/index/uuid_to_id_8db98f0d-bbc2-4d1e-8550-ac87974f942f.pkl  
  inflating: db/chroma-collections.parquet  


In [ ]:
os.environ["OPENAI_API_KEY"] = "sk-..."

In [48]:
persist_directory = 'db'
embedding = OpenAIEmbeddings()

vectordb2 = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding,
                   )

retriever = vectordb2.as_retriever(search_kwargs={"k": 2})

Using embedded DuckDB with persistence: data will be stored in: db


#### Usung turbo GPT API

In [49]:
# Set up the turbo LLM
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

In [50]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=turbo_llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [51]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [52]:
# full example
query = "What are the steps of the Quickstart Guide?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

The Quickstart Guide includes the following sections: Getting Started, Modules, Use Cases, Reference Docs, Ecosystem, and Additional Resources.


Sources:
/Users/erictak/PycharmProjects/langchain/data/content_2.txt
/Users/erictak/PycharmProjects/langchain/data/content_1.txt


In [53]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[0].prompt.template)

Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{context}


In [54]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[1].prompt.template)

{question}
